In [1]:
import os
import sys
import pandas as pd
import re


In [9]:
name_reg = re.compile(r"[^(\w| |\[|\]|\(|\)|-]+")
sys_cat_regex = re.compile(r"[\w\s,-]+")

rogue_words = {'hlist', 'flatlist', 'plainlist', 's', 'br', '',
               'overline', 'list'}

synonyms = [('color', 'colour'), ('luster', 'lustre')]

df = pd.DataFrame()


In [10]:
os.getcwd()

'/Users/chief-blackhood/Desktop/sem 5/IRE/Major_Project/Domain-specific-data-collection/cleaners'

In [11]:
# remove_html = re.compile(r"<.*?/>|<[^>]>.*?</[^>]>")
# bracket_cleaner = re.compile(r"\[|\]")
#
#
# def clean(text):
#     text = remove_html.sub(text, ' ')
#     text = ''.join(bracket_cleaner.split(text))
#     text = ' '.join(text.split())
#     return text

# def clean_sys_cat(pat: str):
#     if pd.isnull(pat):
#         return pat
#     # print(pat)
#     pat = pat.replace('|', ',')
#     content = ''.join([' ' if x.strip() in rogue_words else x for x in sys_cat_regex.findall(pat)])
#     return content

for file in os.listdir('../data/wikipedia'):
    if file.endswith('csv'):
        df = df.append(pd.read_csv(f'../data/wikipedia/{file}'))
        



In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Columns: 601 entries, Unnamed: 0 to area_footnotes
dtypes: float64(79), int64(1), object(521)
memory usage: 6.3+ MB


In [14]:
# df = df.drop(columns=['image', 'caption', 'imagesize'], errors='ignore')
df = df.drop_duplicates(subset=["name"])
df = df.dropna(subset=["name"]).reset_index(drop=True)

In [18]:
# print(df.columns.tolist())
# print('Columns:', len(df.columns))

for (x, y) in synonyms:
    # print(df[[x, y]].info())

    # a = df.index[df[x].notnull()].tolist()
    # b = df.index[df[y].notnull()].tolist()
    # inter = list(set(a).intersection(set(b)))
    df[x] = df[x].combine_first(df[y])
    df = df.drop(columns=[y])

    # print(f"Intersection of {x} and {y}:", len(inter))
    # print(df[[x, y]].iloc[inter])
    # print(df[[x]].info())



In [23]:
print(df.info())

# Attribute wise cleanup here
# df['name'] = df['name'].apply(lambda x: ''.join(name_reg.split(x)))
# df['system'] = df['system'].apply(lambda x: clean_sys_cat(x))
# df['category'] = df['category'].apply(lambda x: clean_sys_cat(x))
#     df['category'] = df['category'].apply(lambda x: clean_sys_cat(x))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1371 non-null   int64 
 1   name           1371 non-null   object
 2   category       1267 non-null   object
 3   image          1120 non-null   object
 4   caption        808 non-null    object
 5   formula        1305 non-null   object
 6   strunz         1198 non-null   object
 7   system         1300 non-null   object
 8   symmetry       1194 non-null   object
 9   unit cell      823 non-null    object
 10  color          1211 non-null   object
 11  cleavage       959 non-null    object
 12  fracture       682 non-null    object
 13  mohs           1129 non-null   object
 14  luster         1130 non-null   object
 15  opticalprop    906 non-null    object
 16  streak         929 non-null    object
 17  gravity        1018 non-null   object
 18  diaphaneity    1033 non-null

In [21]:
df.to_csv('../data/wikipedia/combined_wiki_mineral_raw.csv', index=False)

In [22]:
col_remove_perc = 40.0
min_count = int((col_remove_perc / 100) * df.shape[0] + 1)
df.dropna(axis=1, how='any', thresh=min_count, inplace=True)
df.reset_index(drop=True, inplace=True)

In [24]:
df.to_csv('../data/wikipedia/combined_wiki_mineral_clean.csv', index=False)